### Modified notebook to convert an image dataset with jpeg extension to TFRecords

### Loading packages

In [1]:
import tensorflow as tf
import numpy as np
import os
import scipy.io as spio
from matplotlib import pyplot as plt
from imageio import imread
import cv2

#### Initializations -- setting up image directory and so on
We don't have actual annoptations yet so ignore the we ignore it for now!
When we do actual retraining we need such information.

In [2]:
# define base paths for dataset images
base_dataset_dir = './'
images_folder_name = "images"
annotations_folder_name = "images"
images_dir = os.path.join(base_dataset_dir, images_folder_name)
annotations_dir = os.path.join(base_dataset_dir, annotations_folder_name) 

#### List of all images in the directory

In [3]:
images_filename_list = os.listdir(images_dir)
np.random.shuffle(images_filename_list)
test_images_filename_list = images_filename_list
print("train set size:", len(test_images_filename_list))

train set size: 635


#### Setting up the directory that tfrecords need to be saved

In [4]:
TEST_DATASET_DIR="./deeplab_v3/dataset/tfrecords/"
if not os.path.exists(TEST_DATASET_DIR):
    os.mkdir(TEST_DATASET_DIR)
    
TEST_FILE = 'test.tfrecords'
test_writer = tf.python_io.TFRecordWriter(os.path.join(TEST_DATASET_DIR,TEST_FILE))

In [5]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [6]:
# We are not using this function probably for our project
def read_annotation_from_mat_file(annotations_dir, image_name):
    annotations_path = os.path.join(annotations_dir, (image_name.strip() + ".mat"))
    mat = spio.loadmat(annotations_path)
    img = mat['GTcls']['Segmentation'][0][0]
    return img

### The main function to create TFRecords from image dataset
Here, we also resize the images in the dataset to match the maximum dimension of 513x513 in the pretrained model.

Resizining is done by preserving the aspect ratio of the original images usonhg OpenCV.

In [7]:
def create_tfrecord_dataset(filename_list, writer):
    HEIGHT = 512

    # create training tfrecord
    read_imgs_counter = 0
    for i, image_name in enumerate(filename_list):

        try:
            image_npTMP = imread(os.path.join(images_dir, image_name.strip()))
            ar = HEIGHT / image_npTMP.shape[1]
            dim = (HEIGHT, int(image_npTMP.shape[0] * ar))
            image_np = cv2.resize(image_npTMP, dim)
        except FileNotFoundError:
            print("File:",image_name.strip(),"not found.")
            continue
            
        try:            
            annotation_npTMP = imread(os.path.join(annotations_dir, image_name.strip()))[:,:,0]
            annotation_np_Resize = cv2.resize(annotation_npTMP, dim)
            ret,annotation_np = cv2.threshold(annotation_np_Resize,0.5,1,cv2.THRESH_BINARY)
        except FileNotFoundError:
            print("File:",image_name.strip(),"not found.")
            continue
            
        read_imgs_counter += 1
        image_h = image_np.shape[0]
        image_w = image_np.shape[1]

        img_raw = image_np.tostring()
        annotation_raw = annotation_np.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(image_h),
                'width': _int64_feature(image_w),
                'image_raw': _bytes_feature(img_raw),
                'annotation_raw': _bytes_feature(annotation_raw)}))

        writer.write(example.SerializeToString())
    
    print("End of TfRecord. Total of image written:", read_imgs_counter)
    writer.close()

In [8]:
# create training dataset
create_tfrecord_dataset(test_images_filename_list, test_writer)

File: Icon not found.
End of TfRecord. Total of image written: 634
